# Multi Environment Json Pattern

This pattern allows you to manage configuration for multiple environments.


## Solution Overview

1. The non-sensitive config values should be checked in to the Git so everyone is able to see it.
2. As the project admin, you are the source-of-truth of the config values and you are the only one has permission to deploy config. You should store the non-sensitive config to the repo and sensitive config to a secure place on your local laptop.
3. When you deploy the config, currently you have two config storage options:
    - deploy to [AWS Parameter Store](https://docs.aws.amazon.com/systems-manager/latest/userguide/systems-manager-parameter-store.html)
    - deploy to [AWS S3](https://aws.amazon.com/s3/)
4. Your application runtime can read the config from one of the config storage options above.
5. When you deploy your application, you should store the ``parameter_name`` or ``s3path_config`` information to the environment variable or a static file. So your application can use this information to read the config data from the config storage.

## Sample Usage

### Declare Your Config Schema

In software engineer best practice, declaration and the usage of a Data Model should be separated. Below is the ``config_define.py`` file that defines three things:

1. enumerate all environments you want to use in your project.
2. declare the per environment config data model.
3. subclass from the BaseConfig, this is your main config object.

In [2]:
# content of config_define.py
# -*- coding: utf-8 -*-

import typing as T
import dataclasses

from config_patterns.patterns.multi_env_json import (
    BaseEnvEnum,
    BaseEnv,
    BaseConfig,
)


class EnvEnum(BaseEnvEnum):
    dev = "dev" # development
    int = "int" # integration test
    prod = "prod" # production


@dataclasses.dataclass
class Env(BaseEnv):
    username: T.Optional[str] = dataclasses.field(default=None)
    password: T.Optional[str] = dataclasses.field(default=None)


@dataclasses.dataclass
class Config(BaseConfig):
    @property
    def dev(self) -> Env:
        return self.get_env(EnvEnum.dev)

    @property
    def int(self) -> Env:
        return self.get_env(EnvEnum.int)

    @property
    def prod(self) -> Env:
        return self.get_env(EnvEnum.prod)

    @classmethod
    def get_current_env(cls) -> str:
        return EnvEnum.dev.value


### Read From Local File and Deploy to Config Storage

As the project admin, you need to decide what value to put in the config. So you created two config files ``config.json`` and ``secret_config.json``. You could check in the ``config.json`` to the Git so everyone can see it. But keep the ``secret_config.json`` private, and only give access to people really need it.

In [3]:
# content of config.json
{
    # this config file support comments, you can put documentation in the config file
    "shared": {
        "project_name": "my_project"
    },
    "envs": {
        "dev": {
            "username": "dev.user"
        },
        "int": {
            "username": "int.user"
        },
        "prod": {
            "username": "prod.user"
        }
    }
}

{'shared': {'project_name': 'my_project'},
 'envs': {'dev': {'username': 'dev.user'},
  'int': {'username': 'int.user'},
  'prod': {'username': 'prod.user'}}}

In [4]:
# content of secret-config.json
{
    # this config file support comments, you can put documentation in the config file
    "shared": {
    },
    "envs": {
        "dev": {
            "password": "dev.password"
        },
        "int": {
            "password": "int.password"
        },
        "prod": {
            "password": "prod.password"
        }
    }
}

{'shared': {},
 'envs': {'dev': {'password': 'dev.password'},
  'int': {'password': 'int.password'},
  'prod': {'password': 'prod.password'}}}

Then you can create the config object and deploy it to config storage.

In [5]:
# content of config_deploy.py
# -*- coding: utf-8 -*-

# import from the config_define.py
from config_define import EnvEnum, Env, Config

import os
from pathlib import Path
from boto_session_manager import BotoSesManager
from rich import print as rprint

# Read config from local file
dir_here = Path(os.getcwd())
path_config = str(dir_here.joinpath("config.json"))
path_secret_config = str(dir_here.joinpath("secret_config.json"))


config = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    path_config=path_config,
    path_secret_config=path_secret_config,
)
rprint(config)

Config(
    data={
        'shared': {'project_name': 'my_project'},
        'envs': {
            'dev': {'username': 'dev.user'},
            'int': {'username': 'int.user'},
            'prod': {'username': 'prod.user'}
        }
    },
    secret_data={
        'shared': {},
        'envs': {
            'dev': {'password': 'dev.password'},
            'int': {'password': 'int.password'},
            'prod': {'password': 'prod.password'}
        }
    },
    Env=<class 'config_define.Env'>,
    EnvEnum=<enum 'EnvEnum'>
)

In [6]:
# Deploy config to AWS Parameter Store
bsm = BotoSesManager(profile_name="aws_data_lab_sanhe_us_east_1")
s3dir_config = "s3://669508176277-us-east-1-artifacts/projects/config_pattern/patterns/multi_env_json/"


config.deploy(
    bsm=bsm,
    parameter_with_encryption=True,
)

deploy parameter store for all environment
🚀️ deploy SSM Parameter 'my_project' ...
preview at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project/description?region=us-east-1&tab=Table
parameter data is the same as existing one, do nothing.
done!
🚀️ deploy SSM Parameter 'my_project-dev' ...
preview at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project-dev/description?region=us-east-1&tab=Table
parameter data is the same as existing one, do nothing.
done!
🚀️ deploy SSM Parameter 'my_project-int' ...
preview at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project-int/description?region=us-east-1&tab=Table
parameter data is the same as existing one, do nothing.
done!
🚀️ deploy SSM Parameter 'my_project-prod' ...
preview at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project-prod/description?region=us-east-1&tab=Table
parameter data is the same as existing one, do nothing.
done!

In [7]:
# Deploy config to AWS S3 Store
config.deploy(
    bsm=bsm,
    s3dir_config=s3dir_config,
)

🚀️ deploy config file s3://669508176277-us-east-1-artifacts/projects/config_pattern/patterns/multi_env_json/all.json ...
preview at: https://us-east-1.console.aws.amazon.com/s3/object/669508176277-us-east-1-artifacts?prefix=projects/config_pattern/patterns/multi_env_json/all.json
done!
🚀️ deploy config file s3://669508176277-us-east-1-artifacts/projects/config_pattern/patterns/multi_env_json/dev.json ...
preview at: https://us-east-1.console.aws.amazon.com/s3/object/669508176277-us-east-1-artifacts?prefix=projects/config_pattern/patterns/multi_env_json/dev.json
done!
🚀️ deploy config file s3://669508176277-us-east-1-artifacts/projects/config_pattern/patterns/multi_env_json/int.json ...
preview at: https://us-east-1.console.aws.amazon.com/s3/object/669508176277-us-east-1-artifacts?prefix=projects/config_pattern/patterns/multi_env_json/int.json
done!
🚀️ deploy config file s3://669508176277-us-east-1-artifacts/projects/config_pattern/patterns/multi_env_json/prod.json ...
preview at: https

### Use Your Config in Application Code

In your application code, you could create the config object by reading the config storage. Then use the Python config object to access those config values.

In [8]:
# content of config_init.py
# -*- coding: utf-8 -*-

from config_define import EnvEnum, Env, Config

from rich import print as rprint
from boto_session_manager import BotoSesManager

# create boto session manager object for AWS SDK authentication
bsm = BotoSesManager(profile_name="aws_data_lab_sanhe_us_east_1")
parameter_name = "my_project-dev"
s3dir_config = "s3://669508176277-us-east-1-artifacts/projects/config_pattern/patterns/multi_env_json/"

You could read the config from AWS Parameter Store. For security reason, assuming that you want to load the "dev" config, you won't be able to access any of the "prod" config from your application runtime.

In [9]:
config = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    parameter_name=parameter_name,
    parameter_with_encryption=True,
)
rprint(config)
rprint(config.dev)
rprint(f"config.dev.username = {config.dev.username!r}")
rprint(f"config.dev.password = {config.dev.password!r}")

Config(
    data={'shared': {'project_name': 'my_project'}, 'envs': {'dev': {'username': 'dev.user'}}},
    secret_data={'shared': {}, 'envs': {'dev': {'password': 'dev.password'}}},
    Env=<class 'config_define.Env'>,
    EnvEnum=<enum 'EnvEnum'>
)

Env(project_name='my_project', env_name='dev', username='dev.user', password='dev.password')

config.dev.username = 'dev.user'

config.dev.password = 'dev.password'

In [10]:
# You can NOT access prod config from dev environment
config.prod

KeyError: 'prod'

You could also read the config from AWS S3.

In [11]:
config = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    s3path_config=f"{s3dir_config}prod.json",
)
rprint(config)
rprint(config.prod)
rprint(f"config.prod.username = {config.prod.username!r}")
rprint(f"config.prod.password = {config.prod.password!r}")

Config(
    data={'shared': {'project_name': 'my_project'}, 'envs': {'prod': {'username': 'prod.user'}}},
    secret_data={'shared': {}, 'envs': {'prod': {'password': 'prod.password'}}},
    Env=<class 'config_define.Env'>,
    EnvEnum=<enum 'EnvEnum'>
)

Env(project_name='my_project', env_name='prod', username='prod.user', password='prod.password')

config.prod.username = 'prod.user'

config.prod.password = 'prod.password'

## Delete Config from Config Storage

At the end, you can delete all config from config storage.

In [13]:
config = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    path_config=path_config,
    path_secret_config=path_secret_config,
)

In [14]:
config.delete(
    bsm=bsm,
    use_parameter_store=True,
)

delete parameter store for all environment
🗑️ delete SSM Parameter 'my_project' ...
verify at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project/description?region=us-east-1&tab=Table
done!
🗑️ delete SSM Parameter 'my_project-dev' ...
verify at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project-dev/description?region=us-east-1&tab=Table
done!
🗑️ delete SSM Parameter 'my_project-int' ...
verify at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project-int/description?region=us-east-1&tab=Table
done!
🗑️ delete SSM Parameter 'my_project-prod' ...
verify at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project-prod/description?region=us-east-1&tab=Table
done!


In [15]:
config.delete(
    bsm=bsm,
    s3dir_config=s3dir_config,
)

🗑️ delete config file s3://669508176277-us-east-1-artifacts/projects/config_pattern/patterns/multi_env_json/all.json ...
preview at: https://us-east-1.console.aws.amazon.com/s3/object/669508176277-us-east-1-artifacts?prefix=projects/config_pattern/patterns/multi_env_json/all.json
done!
🗑️ delete config file s3://669508176277-us-east-1-artifacts/projects/config_pattern/patterns/multi_env_json/dev.json ...
preview at: https://us-east-1.console.aws.amazon.com/s3/object/669508176277-us-east-1-artifacts?prefix=projects/config_pattern/patterns/multi_env_json/dev.json
done!
🗑️ delete config file s3://669508176277-us-east-1-artifacts/projects/config_pattern/patterns/multi_env_json/int.json ...
preview at: https://us-east-1.console.aws.amazon.com/s3/object/669508176277-us-east-1-artifacts?prefix=projects/config_pattern/patterns/multi_env_json/int.json
done!
🗑️ delete config file s3://669508176277-us-east-1-artifacts/projects/config_pattern/patterns/multi_env_json/prod.json ...
preview at: https